<a href="https://colab.research.google.com/github/Jimmynycu/ATS_with_gemini_api/blob/main/Applicant_Tracking_System_with_ollama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q pyautogen pypdf pandas openpyxl python-dotenv autogen --quiet

!curl -fsSL https://ollama.com/install.sh | sh

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 834.0/834.0 kB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.1/119.1 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 13.9 MB/s eta 0:00:00
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [2]:
import autogen
import json
import pandas as pd
from pypdf import PdfReader
import os
import asyncio

In [118]:
pid = os.fork()
if pid == 0:
    os.execv('/usr/local/bin/ollama', ['ollama', 'serve'])
else:
    print("Ollama server started in the background.")

await asyncio.sleep(5)
!ollama pull llama3:instruct &

/tmp/ipython-input-1792677764.py:1: DeprecationWarning: This process (pid=1140) is multi-threaded, use of fork() may lead to deadlocks in the child.
  pid = os.fork()


Ollama server started in the background.



In [119]:
import httpx
import json # Import json to potentially print the response data

ollama_chat_url = "http://127.0.0.1:11434/api/chat"
model_name = "llama3:instruct"
prompt_message = {"role": "user", "content": "Are you ready?"}
payload = {
    "model": model_name,
    "messages": [prompt_message],
    "stream": False
}

print(f"Sending simple test prompt to {ollama_chat_url}...")

# Send the POST request
response = httpx.post(ollama_chat_url, json=payload, timeout=10.0) # Added a basic timeout

# Print the status code and response text/json
print(f"Response Status Code: {response.status_code}")
try:
    # Attempt to parse and print JSON if possible
    print("Response Body (JSON if available):")
    print(json.dumps(response.json(), indent=2))
except json.JSONDecodeError:
    # Otherwise, print raw text
    print("Response Body (Text):")
    print(response.text)

print("\nSimple test complete.")

Sending simple test prompt to http://127.0.0.1:11434/api/chat...
Response Status Code: 200
Response Body (JSON if available):
{
  "model": "llama3:instruct",
  "created_at": "2025-09-09T19:48:09.853287692Z",
  "message": {
    "role": "assistant",
    "content": "I'm always ready! What's next?"
  },
  "done_reason": "stop",
  "done": true,
  "total_duration": 3172958254,
  "load_duration": 2702521389,
  "prompt_eval_count": 14,
  "prompt_eval_duration": 151055268,
  "eval_count": 10,
  "eval_duration": 318451398
}

Simple test complete.


In [52]:
config_list=[
    {
        "model": "llama3:instruct",
        "base_url": "http://127.0.0.1:11434/v1",
        "api_key": "ollama"
    }
]

In [53]:
# --- Configuration ---
llm_config = {
        "config_list": config_list,
        "temperature": 0.0
        }

In [71]:
parser_agent = autogen.ConversableAgent(
    name="ResumeParserAgent",
    system_message="""You are an expert HR data extraction specialist.
            Your ONLY task is to parse resume text into a structured JSON format. You must STRICTLY and LITERALLY adhere to the following rules:
            1.  Begin your response with a ```json code block.
            2.  Provide the complete, raw JSON object inside the code block.
            3.  End your response with the closing ``` code block.
            4.  Your entire response must consist of NOTHING but the JSON object enclosed within the opening and closing ```json``` code blocks.
            5.  DO NOT add any other text, commentary, greetings, or explanations before, after, or inside the code block.
            """,
            llm_config=llm_config,
            human_input_mode="NEVER",
    )

# Function to check for the termination condition
def is_json_termination_msg(message):
    content = message.get("content", "")
    print(f"Checking message for termination: {content}")
    # This checks for a code block that starts with "```json" and ends with "```".
    return content.startswith("```json") and content.endswith("```")

user_proxy = autogen.UserProxyAgent(
    name="UserProxyAgent",
    human_input_mode="NEVER",
    code_execution_config={"work_dir": "coding", "use_docker": False},
    is_termination_msg=is_json_termination_msg,
    max_consecutive_auto_reply=10,
)

In [99]:
# PDF
pdf_path = "Jimmy_CV.pdf"
raw_resume_text = ""
try:
    reader = PdfReader(pdf_path)
    for page in reader.pages:
        raw_resume_text += page.extract_text() or ""
    print("PDF text extracted successfully.")
except FileNotFoundError:
    print(f"Error: The file '{pdf_path}' not found.")
else:
    # --- Agentic Workflow ---
    user_proxy.initiate_chat(
        parser_agent,
        message=f"""
            Please parse the following resume text into the required JSON format.

            JSON Schema:
            {{
              "contact_information": {{ "name": "string", "email": "string", "phone": "string", "location": "string", "links": ["string"] }},
              "professional_summary": "string",
              "skills": ["string"],
              "work_experience": [ {{ "job_title": "string", "company": "string", "location": "string", "responsibilities": ["string"] }} ],
              "education": [ {{ "degree": "string", "institution": "string", "location": "string" }} ],
              "personal_projects": [ {{ "project_name": "string", "description": "string" }} ]
            }}

            Resume Text:
            ---
            {raw_resume_text}
            ---
            """,
    )

    # --- Save Output ---
    json_string = None
    last_message = user_proxy.last_message(parser_agent)

    if last_message and "```json" in last_message["content"]:
        json_string = last_message["content"].split("```json")[1].split("```")[0].strip()

    if json_string:
        print("\n LLM generated the structured JSON. Now saving to Excel...")
        try:
            data = json.loads(json_string)
            with pd.ExcelWriter("resume_data.xlsx", engine='openpyxl') as writer:

                # 1. Save Contact Information
                contact_info_df = pd.DataFrame([data.get('contact_information', {})])
                contact_info_df.to_excel(writer, sheet_name='Contact_Info', index=False)

                # 2. Save Professional Summary
                professional_summary_df = pd.DataFrame({'summary': [data.get('professional_summary', '')]})
                professional_summary_df.to_excel(writer, sheet_name='Summary', index=False)

                # 3. Save Skills
                skills_list = data.get('skills', [])
                skills_df = pd.DataFrame(skills_list, columns=['Skill'])
                skills_df.to_excel(writer, sheet_name='Skills', index=False)

                # 4. Save Work Experience
                work_experience_list = data.get('work_experience', [])
                work_experience_df = pd.DataFrame(work_experience_list)
                work_experience_df.to_excel(writer, sheet_name='Work_Experience', index=False)

                # 5. Save Education
                education_list = data.get('education', [])
                education_df = pd.DataFrame(education_list)
                education_df.to_excel(writer, sheet_name='Education', index=False)

                # 6. Save Personal Projects
                personal_projects_list = data.get('personal_projects', [])
                personal_projects_df = pd.DataFrame(personal_projects_list)
                personal_projects_df.to_excel(writer, sheet_name='Personal_Projects', index=False)

            print("Excel file 'resume_data.xlsx' saved successfully. You can find it in the file browser.")

        except (json.JSONDecodeError, KeyError) as e:
            print(f"Error: Could not process the JSON from the LLM. Error: {e}")
    else:
        print("\nError: Could not find the JSON output from the LLM conversation.")

PDF text extracted successfully.
UserProxyAgent (to ResumeParserAgent):


            Please parse the following resume text into the required JSON format.

            JSON Schema:
            {
              "contact_information": { "name": "string", "email": "string", "phone": "string", "location": "string", "links": ["string"] },
              "professional_summary": "string",
              "skills": ["string"],
              "work_experience": [ { "job_title": "string", "company": "string", "location": "string", "responsibilities": ["string"] } ],
              "education": [ { "degree": "string", "institution": "string", "location": "string" } ],
              "personal_projects": [ { "project_name": "string", "description": "string" } ]
            }

            Resume Text:
            ---
            JIMMY LIU
Taiwan Taipei⋄ +886 978768367 ⋄ jimmyliu.ii12@nycu.edu.tw ⋄ Github: Jimmynyu
PROFESSIONAL SUMMARY
AI/ML engineer with specialized expertise in LLM quantization and foun

ResumeParserAgent (to UserProxyAgent):

```json
{
  "contact_information": {
    "name": "JIMMY LIU",
    "email": "jimmyliu.ii12@nycu.edu.tw",
    "phone": "+886 978768367",
    "location": "Taiwan Taipei",
    "links": ["Github: Jimmynyu"]
  },
  "professional_summary": "AI/ML engineer with specialized expertise in LLM quantization and foundation models. Seeking an opportunity to contribute to AI development, leveraging LLM optimization and deployment background.",
  "skills": [
    "Agent Coding",
    "Python (Advanced)",
    "PyTorch & TensorFlow",
    "Git version control",
    "LLM fine-tuning & deployment",
    "Foundation model dev.",
    "Deep learning optimization",
    "Quantitative analysis",
    "Linux administration",
    "Verilog",
    "Communication",
    "English TOEIC 935"
  ],
  "work_experience": [
    {
      "job_title": "EDA Software Engineer",
      "company": "MediaTek",
      "location": "Hsinchu, Taiwan",
      "responsibilities": [
        "Implemented and o

In [114]:
def load_data_from_excel(file_path: str):
    if not os.path.exists(file_path):
        print(f"Error: The file '{file_path}' was not found.")
        return None
    print(f" Loading structured data from '{file_path}'...")
    xls = pd.ExcelFile(file_path)
    data = {}
    data['contact_information'] = pd.read_excel(xls, 'Contact_Info').fillna('').to_dict('records')[0]
    data['professional_summary'] = pd.read_excel(xls, 'Summary').fillna('').to_dict('records')[0]['summary']
    data['skills'] = pd.read_excel(xls, 'Skills')['Skill'].dropna().tolist()
    data['work_experience'] = pd.read_excel(xls, 'Work_Experience').fillna('').to_dict('records')
    data['education'] = pd.read_excel(xls, 'Education').fillna('').to_dict('records')
    data['personal_projects'] = pd.read_excel(xls, 'Personal_Projects').fillna('').to_dict('records')

    print("Data loaded and reconstructed.")
    return data

def get_job_description():
    return """
    About Us: TSMC IT Business AI team is dedicated to developing next-generation technologies...
    Requirements:
    * A minimum of a Master's degree in Computer Science, Artificial Intelligence...
    * At least 3 years of experience in AI/Machine Learning. Strong proficiency in Python with a proven track record in AI/ML project development. Experience with Natural Language Processing (NLP) and Large Language Models (LLMs), as well as graph embedding techniques...
    * Excellent communication and teamwork skills.
    * Business domain knowledge in supply chain management is desirable.
    """


In [123]:
# --- Agent Definitions ---
recruiter_agent = autogen.ConversableAgent(
    name="RecruiterAgent",
    system_message="""You are a senior technical recruiter. Your task is to analyze a candidate's resume against a job description.
            Think carefully about your response. Your entire response must be a JSON object inside a ```json code block.
            Follow these steps exactly:
            1.  Generate a single JSON object that conforms to the schema provided below.
            2.  Place the opening JSON code block delimiter: ```json
            3.  Paste the generated JSON object on the next line.
            4.  Place the closing code block delimiter: ```
            5.  Your response must contain **nothing** but this JSON code block. Do not add any conversational text or explanations before or after it.

            JSON Schema:
            {
              "candidate_name": "string",
              "match_score": "integer",
              "summary": "string",
              "keyword_match": { "matched": ["string"], "missing": ["string"] }
            }""",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

def is_json_termination_msg(message):
    """Checks if the message content contains a JSON code block."""
    content = message.get("content", "")
    print(f"Checking message for termination: {content}")
    return "```json" in content

user_proxy = autogen.UserProxyAgent(
    name="UserProxyAgent",
    human_input_mode="NEVER",
    is_termination_msg=is_json_termination_msg,  # Add the termination condition
    max_consecutive_auto_reply=2,
)


In [124]:
# --- Agentic Workflow ---
candidate_data = load_data_from_excel("resume_data.xlsx")
if candidate_data:
    job_desc = get_job_description()
    user_proxy.initiate_chat(
        recruiter_agent,
        message=f"""Please analyze the following candidate's resume data against the job description.
            Job Description:
            ---
            {job_desc}
            ---

            Candidate's Resume Data (JSON):
            ---
            {json.dumps(candidate_data, indent=2)}
            ---
            """,
    )

    # --- Print Final Output ---
    final_json_response = None
    last_message = user_proxy.last_message(recruiter_agent)
    if "```json" in last_message["content"]:
        final_json_response = last_message["content"].split("```json")[1].split("```")[0].strip()

    if final_json_response:
        print("\n" + "="*20 + " Candidate Ranking Analysis " + "="*20)
        try:
            analysis_data = json.loads(final_json_response)
            print(json.dumps(analysis_data, indent=2))
            print("\nFinished. ATS workflow complete.")
        except json.JSONDecodeError:
            print("Error: Could not parse the JSON from the LLM's final response.")
    else:
        print("\nError: Could not find the final JSON analysis in the conversation.")

📊 Loading structured data from 'resume_data.xlsx'...
✅ Data loaded and reconstructed.
UserProxyAgent (to RecruiterAgent):

Please analyze the following candidate's resume data against the job description.
            Job Description:
            ---
            
    About Us: TSMC IT Business AI team is dedicated to developing next-generation technologies...
    Requirements:
    * A minimum of a Master's degree in Computer Science, Artificial Intelligence...
    * At least 3 years of experience in AI/Machine Learning. Strong proficiency in Python with a proven track record in AI/ML project development. Experience with Natural Language Processing (NLP) and Large Language Models (LLMs), as well as graph embedding techniques...
    * Excellent communication and teamwork skills.
    * Business domain knowledge in supply chain management is desirable.
    
            ---

            Candidate's Resume Data (JSON):
            ---
            {
  "contact_information": {
    "name": "JIMM

RecruiterAgent (to UserProxyAgent):

```json
{
  "candidate_name": "JIMMY LIU",
  "match_score": 85,
  "summary": "The candidate has a strong background in AI/ML, with experience in LLM and NLP. They have a proven track record of developing AI projects and have excellent communication skills.",
  "keyword_match": {
    "matched": [
      "Python", 
      "AI/Machine Learning", 
      "Natural Language Processing (NLP)", 
      "Large Language Models (LLMs)", 
      "graph embedding techniques"
    ],
    "missing": [
      "Master's degree in Computer Science, Artificial Intelligence", 
      "At least 3 years of experience in AI/Machine Learning", 
      "Business domain knowledge in supply chain management"
    ]
  }
}

--------------------------------------------------------------------------------
Checking message for termination: ```json
{
  "candidate_name": "JIMMY LIU",
  "match_score": 85,
  "summary": "The candidate has a strong background in AI/ML, with experience in LLM and 